In [ ]:
# ========================================
# Cell 1: 라이브러리 import 및 설정
# ========================================

import os
import sys
import pandas as pd
sys.path.append(os.path.abspath('../src'))

from processors._pdf_text_extractor import PDFTextExtractor
from processors._pdf_pair_matcher import PDFPairMatcher
from processors._translation_pair_generator import TranslationPairGenerator
from config import PROCESSED_DIR



In [ ]:

# ========================================
# Cell 2: PDF 페어 찾기
# ========================================

print("🔍 PDF 페어를 찾는 중...")

matcher = PDFPairMatcher()
pdf_pairs = matcher.find_pdf_pairs()

if pdf_pairs:
    print(f"✅ {len(pdf_pairs)}개의 PDF 페어를 발견했습니다.")
    for en_path, ko_path in pdf_pairs[:3]:  # 처음 3개만 표시
        print(f"   - {os.path.basename(ko_path)}")
    if len(pdf_pairs) > 3:
        print(f"   ... 외 {len(pdf_pairs)-3}개")
else:
    print("❌ 매칭되는 PDF 페어를 찾을 수 없습니다.")




In [ ]:
# ========================================
# Cell 3: 텍스트 추출 및 번역 쌍 생성
# ========================================

if not pdf_pairs:
    print("❌ 처리할 PDF 페어가 없습니다.")
else:
    print("\n📖 텍스트 추출 및 번역 쌍 생성을 시작합니다...")
    
    extractor = PDFTextExtractor(min_text_length=20)
    generator = TranslationPairGenerator(similarity_threshold=0.7)
    
    all_pairs = []
    
    for i, (en_path, ko_path) in enumerate(pdf_pairs):
        print(f"\n[{i+1}/{len(pdf_pairs)}] 처리 중: {os.path.basename(ko_path)}")
        
        try:
            # 텍스트 추출
            en_segments = extractor.extract_segments_from_pdf(en_path)
            ko_segments = extractor.extract_segments_from_pdf(ko_path)
            
            print(f"   - 영어: {len(en_segments)}개, 한국어: {len(ko_segments)}개 세그먼트")
            
            # 번역 쌍 생성
            pairs = generator.generate_pairs(en_segments, ko_segments, ko_path)
            all_pairs.extend(pairs)
            
            print(f"   - ✅ {len(pairs)}개 번역 쌍 생성")
            
        except Exception as e:
            print(f"   - ❌ 오류: {e}")
    
    print(f"\n🎉 총 {len(all_pairs)}개의 번역 쌍을 생성했습니다!")




In [ ]:
# ========================================
# Cell 4: 결과 저장 및 미리보기
# ========================================

if all_pairs:
    # 결과 저장
    csv_path = os.path.join(PROCESSED_DIR, "pdf_translation_pairs.csv")
    
    df = pd.DataFrame(all_pairs)
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')
    
    print(f"💾 결과 저장 완료: {csv_path}")
    print(f"📊 평균 유사도: {df['similarity_score'].mean():.3f}")
    print(f"📄 소스 파일 수: {df['source_file'].nunique()}개")
    
    # 미리보기
    print("\n🔍 상위 3개 번역 쌍 미리보기:")
    print("-" * 80)
    
    top_pairs = df.nlargest(3, 'similarity_score')
    
    for i, (_, row) in enumerate(top_pairs.iterrows(), 1):
        print(f"\n[{i}] 유사도: {row['similarity_score']:.3f}")
        print(f"🇺🇸 {row['source_text'][:80]}{'...' if len(row['source_text']) > 80 else ''}")
        print(f"🇰🇷 {row['target_text'][:80]}{'...' if len(row['target_text']) > 80 else ''}")
        
else:
    print("⚠️ 저장할 번역 쌍이 없습니다.")